In [1]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!pip3 install tokenizers wandb sentencepiece triton

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 10.9 MB/s eta 0:00:00


In [3]:
!pip3 install transformers huggingface-hub tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.8 MB/s eta 0:00:00


In [4]:
!pip3 install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [5]:
import os

os.chdir("drive/")
os.chdir('My Drive')
os.chdir('Research')
os.chdir('LLM')
os.chdir('TinyLLM')

In [6]:
OUTPUT_DIR = './outputs/'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [7]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Dec 28 06:00:53 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              44W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

# CFG

In [8]:
# ====================================================
# CFG
# ====================================================
class CFG:
    wandb=True
    competition='PPPM'
    _wandb_kernel='bluehills'
    debug=False
    apex=True
    print_freq=100
    num_workers=4
    model="bert-base-uncased"
    scheduler='cosine' # ['linear', 'cosine']
    batch_scheduler=True
    num_cycles=0.5
    num_warmup_steps=50 # [0, 50, 100]
    epochs=5
    encoder_lr=1e-5 #2e-5
    decoder_lr=1e-5 #2e-5
    min_lr=5e-7
    eps=5e-7
    betas=(0.9, 0.999)
    batch_size=16
    fc_dropout=0.15
    target_size=1
    max_len=512
    weight_decay=0.01
    gradient_accumulation_steps=1
    max_grad_norm=1000
    seed=44
    train_all_index=20
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
    train=True

if CFG.debug:
    CFG.epochs = 2
    CFG.trn_fold = [0]

In [ ]:
# ====================================================
# wandb
# ====================================================
if CFG.wandb:
    import wandb
    try:
        # from kaggle_secrets import UserSecretsClient
        # user_secrets = UserSecretsClient()
        # secret_value_0 = user_secrets.get_secret("wandb_api")
        # wandb.login(key=secret_value_0)
        print('login to wandb')
        wandb.login()
        anony = None
    except:
        anony = "must"
        print('If you want to use your W&B account, go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as wandb_api. \nGet your W&B access token from here: https://wandb.ai/authorize')


    def class2dict(f):
        return dict((name, getattr(f, name)) for name in dir(f) if not name.startswith('__'))

    run = wandb.init(project='TinyLLM-Train',
                     name=CFG.model,
                     config=class2dict(CFG),
                     group=CFG.model,
                     job_type="train",
                     anonymous=anony)

login to wandb


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: bluehills. Use `wandb login --relogin` to force relogin


# Library

In [9]:
# ====================================================
# Library
# ====================================================
import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math
from math import sqrt
from dataclasses import dataclass
from typing import Optional, Tuple
import shutil
import string
import pickle
import random
import joblib
import itertools
import logging
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedGroupKFold

import torch
print(f"torch.__version__: {torch.__version__}")
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset
import torch.cuda.amp as amp
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group
import torch.distributed as dist

import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig, AutoModelForSequenceClassification
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
from transformers import AutoTokenizer, AutoConfig, AutoModelForTokenClassification
import tiktoken

import datasets
import huggingface_hub
import matplotlib.font_manager as font_manager
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats

%env TOKENIZERS_PARALLELISM=true

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torch.__version__: 2.5.1+cu121
tokenizers.__version__: 0.21.0
transformers.__version__: 4.47.1
env: TOKENIZERS_PARALLELISM=true


# Utils

In [10]:
# ====================================================
# Utils
# ====================================================

def get_logger(filename=OUTPUT_DIR+'train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(seed=CFG.seed)

## Model Args

In [11]:
@dataclass
class ModelArgs:
    dim: int = 768 #256 #128
    n_layers: int = 12 # Llama 3 8b uses 32
    n_heads: int = 12 # Llama 3 8b uses 32
    n_kv_heads: Optional[int] = 4 # Llama 3 8b's uses 8
    vocab_size: int = 50304 #512 # Llama 3 uses a more complicated tokenizer of length 128256
    multiple_of: int = 256  # make SwiGLU hidden layer size multiple of large power of 2. Llama 3 8b's uses 1024
    ffn_dim_multiplier: Optional[float] = None # Llama 3 8b's uses 1.3, which changes the ending hidden_dim slightly
    norm_eps: float = 1e-5
    rope_theta: float = 10000 # Llama 3 8b uses 500000
    max_batch_size: int = 16
    grad_accum_steps: int = 4
    max_seq_len: int = 1024 #256 # Llama 3 8b trained with 8192 but their maximum kv cache chunk size during inference is 2048
    dropout_rate: float = 0.1
    bfloat_supported: bool = True #does the machine support bfloat16 for autocast
    flash_attn_supported: bool = False #does the mahine support Flahs Attention?


## SLM Helper

In [12]:
class RMSNorm(torch.nn.Module):
    def __init__(self, dim: int, eps: float = 1e-6):
        super().__init__()
        self.eps = eps
        self.weight = nn.Parameter(torch.ones(dim))

    def _norm(self, x):
        return x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + self.eps)

    def forward(self, x):
        output = self._norm(x.float()).type_as(x)
        return output * self.weight

In [13]:
def create_causal_mask(seq_len: int, device: str):
    mask = torch.full(
      (seq_len, seq_len),
      float("-inf")
    )
    mask = torch.triu(mask, diagonal=1)
    return(mask.to(device))


def precompute_rotary_embeddings(head_dim: int, seq_len: int, device: str, theta: float = 10000.0):
    #default theta set to 10,000. recomended value

    #STEP 1: Compute the theta_i values
    #formula for theta_i(freqs) is theta ^ (-(2(i - 1)/dim)) where i = 1, 2, ... dim/2
    #torch.arrange gived number from 0 to end integer

    freqs = 1.0 / (theta ** (torch.arange(0, head_dim, 2)[: (head_dim // 2)].float() / head_dim))
    #freqs shape will be size head_dim/2

    #print(f'freqs: {freqs.shape}\n{freqs}\n')

    #STEP 2: Compute the m_i * theta_i matrix
    #just an array of numbers from 0 to 2xseq_length
    # why 2xseq_length? - so we can project into the generated tokens
    t = torch.arange(seq_len, dtype=torch.float32)
    #print(f't: {t.shape}\n{t}\n')

    freqs = torch.outer(t, freqs)   #outer product of tensors
    #print(f'freqs: {freqs.shape}\n{freqs}\n')
    #this shape will be a matrix of 2xseq_length, head_dim

    # STEP 3: return the cosing and sine values for the m_i * theta_i computed earlier
    # torch.ones_like(freqs)  #returns tensor of ones with same shape as freqs
    # torch.polar(abs, angle) - returns a complex tensor - abs⋅cos(angle)+abs⋅sin(angle)⋅j

    freqs_complex = torch.polar(torch.ones_like(freqs), freqs)[:seq_len]
    #print(f'freqs_complex: {freqs_complex.shape}\n{freqs_complex}')

    return(freqs_complex.to(device))


def apply_rotary_embeddings(xq: torch.Tensor, xk: torch.Tensor, freqs_cis: torch.Tensor, ) -> Tuple[torch.Tensor, torch.Tensor]:
    #STEP 1: group pairs of the q and k tensors and move them into complex numbers e.f. [x1, x2, x3, x4] => [[x1 + ix2], [x3 + ix4]]
    # we only apply the ROPE Embeddings to the q and k matrices, not the v matrix
    xq_ = torch.view_as_complex(xq.float().reshape(*xq.shape[:-1], -1, 2))
    xk_ = torch.view_as_complex(xk.float().reshape(*xk.shape[:-1], -1, 2))

    #STEP 2: reshape the precomputed ROPE Embeddings into a form we can multiply with
    ndim = xq_.ndim
    shape = [d if i == 1 or i == ndim - 1 else 1 for i, d in enumerate(xq_.shape)]
    freqs_cis = freqs_cis.view(*shape)

    #STEP 3: Multiply the reshpaed q and k matrices and reshape back into the original shape
    xq_out = torch.view_as_real(xq_ * freqs_cis).flatten(3)
    xk_out = torch.view_as_real(xk_ * freqs_cis).flatten(3)

    return xq_out.type_as(xq), xk_out.type_as(xk)

## Model

In [14]:
params = ModelArgs()

In [15]:
class Attention(nn.Module):
    def __init__(self, args: ModelArgs, device):
        super().__init__()
        self.n_heads = args.n_heads
        self.n_kv_heads = args.n_heads if args.n_kv_heads is None else args.n_kv_heads
        self.dim = args.dim
        self.head_dim = args.dim // args.n_heads

        attn_shape = (self.n_heads + 2 * args.n_kv_heads) * self.head_dim

        # Attention Weight Matrix
        self.qkv_attn = nn.Linear(args.dim, attn_shape, bias=False)
        #Output Weight Matrix
        self.wo = nn.Linear(args.n_heads * self.head_dim, args.dim, bias=False)


        self.cache_k = torch.zeros(
            (args.max_batch_size, args.max_seq_len, self.n_kv_heads, self.head_dim),
            requires_grad = False
        ).to(device)
        self.cache_v = torch.zeros(
            (args.max_batch_size, args.max_seq_len, self.n_kv_heads, self.head_dim),
            requires_grad = False
        ).to(device)


    def forward(self,  x: torch.Tensor,  freqs_cis: torch.Tensor, mask: Optional[torch.Tensor], start_pos: int = None,  ):
        num_batches, seqlen, _ = x.shape
        q_per_kv = self.n_heads // self.n_kv_heads


        # e.g. for dim = 768, num_q_heads = 12, num_kv_heads = 4
        # this would create a matix with dimensions B, T, (768 + 256 + 256)
        qkv = self.qkv_attn(x)

        # create a view with B, T, 4, 5, 64, (768/12)
        qkv = qkv.view(num_batches, seqlen, self.n_kv_heads, q_per_kv + 2, self.head_dim)

        #now split
        q, k, v = qkv.split((q_per_kv, 1, 1), dim=-2)

        q = q.reshape(num_batches, seqlen, -1, self.head_dim)  # (B, T, nh_q, hs)
        k = k.reshape(num_batches, seqlen, -1, self.head_dim)
        v = v.reshape(num_batches, seqlen, -1, self.head_dim)

        q, k = apply_rotary_embeddings(q, k, freqs_cis=freqs_cis)

        if start_pos is not None:
            #KV CACHE FOR INFERENCE
            #this is from the LLAMA3 code
            # make sure our cache is on the right device
            self.cache_k = self.cache_k.to(q)
            self.cache_v = self.cache_v.to(q)

            # set the values in our cache according to the current input
            self.cache_k[:num_batches, start_pos : start_pos + seqlen] = k
            self.cache_v[:num_batches, start_pos : start_pos + seqlen] = v

            # grab our key and value matrixes which have a longer sequence length than our queries
            keys = self.cache_k[:num_batches, : start_pos + seqlen]
            values = self.cache_v[:num_batches, : start_pos + seqlen]
        else:
            # TRAINING
            keys, values = k, v

        scale = 1/q.shape[-1]**0.5
        q = q.transpose(1, 2)
        k = k.transpose(1, 2)
        v = v.transpose(1, 2)
        if q.size() != k.size():
            k = k.repeat_interleave(q.shape[1]//k.shape[1], dim=1)
            v = v.repeat_interleave(q.shape[1]//v.shape[1], dim=1)
        #automatically uses flash attention and applies softmax scale
        o = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=mask, dropout_p=0.0, is_causal=mask is None)
        #o = o.transpose(1,2).contiguous().view(num_batches, seqlen, self.dim)
        o = o.transpose(1,2).reshape(num_batches, seqlen, self.dim)

        return self.wo(o)

In [16]:
class FeedForward(nn.Module):
    def __init__(self, dim: int, hidden_dim: int, multiple_of: int, ffn_dim_multiplier: Optional[float],):
        super().__init__()
        # custom dim factor multiplier that ensures we're using a multiple of 256, likely for hardware efficiency reasons
        hidden_dim = int(2 * hidden_dim / 3)
        if ffn_dim_multiplier is not None:
            hidden_dim = int(ffn_dim_multiplier * hidden_dim)
        hidden_dim = multiple_of * ((hidden_dim + multiple_of - 1) // multiple_of)

        self.w1 = nn.Linear(dim, hidden_dim, bias=False)
        self.w2 = nn.Linear(hidden_dim, dim, bias=False)
        self.w3 = nn.Linear(dim, hidden_dim, bias=False)

    def forward(self, x):
        return self.w2(F.silu(self.w1(x)) * self.w3(x))


#Repeating Block
class TransformerBlock(nn.Module):
    def __init__(self, layer_id: int, args: ModelArgs, device):
        super().__init__()
        self.n_heads = args.n_heads
        self.dim = args.dim
        self.head_dim = args.dim // args.n_heads
        self.attention = Attention(args, device)
        self.feed_forward = FeedForward(
            dim=args.dim,
            hidden_dim=4 * args.dim,
            multiple_of=args.multiple_of,
            ffn_dim_multiplier=args.ffn_dim_multiplier,
        )
        self.layer_id = layer_id
        self.attention_norm = RMSNorm(args.dim, eps=args.norm_eps)
        self.ffn_norm = RMSNorm(args.dim, eps=args.norm_eps)
        self.dropout_rate = args.dropout_rate

    def forward(self, x: torch.Tensor, freqs_cis: torch.Tensor, mask: Optional[torch.Tensor], start_pos: int = None, training = False, ):
        # our two residual connections, plus dropout which will only happen if we're training
        #RESIDUAL CONNECTION 1
        h = x # save the inputs for the residual connection
        #apply RMS Norm to the inputs
        x = self.attention_norm(x)
        #apply the attention layer
        x = self.attention(x, freqs_cis, mask, start_pos)
        #Apply the Dropout ONLY IF WE'RE IN THE TRAINING LOOP
        x = F.dropout(x, p=self.dropout_rate, training=training)
        #make the residual connection
        h = h + x

        #RESIDUAL CONNECTION 1
        out = h # save the inputs for the residual connection
        #apply RMS Norm to the inputs
        h = self.ffn_norm(h)
        #apply the feed forward layer
        h = self.feed_forward(h)
        #Apply the Dropout ONLY IF WE'RE IN THE TRAINING LOOP
        h = F.dropout(h, p=self.dropout_rate, training=training)
        #make the residual connection
        out = out + h

        return out

In [17]:
class LLM(nn.Module):
    def __init__(self, params: ModelArgs, tokenizer, device):
        super().__init__()
        self.params = params
        self.vocab_size = params.vocab_size
        self.n_layers = params.n_layers
        self.max_seq_len = params.max_seq_len
        self.tokenizer = tokenizer
        self.device = device

        self.tok_embeddings = nn.Embedding(params.vocab_size, params.dim)

        self.layers = torch.nn.ModuleList()
        for layer_id in range(params.n_layers):
            self.layers.append(TransformerBlock(layer_id, params, device))

        # final norm and linear layer
        self.norm = RMSNorm(params.dim, eps=params.norm_eps)
        self.output = nn.Linear(
            params.dim,
            params.vocab_size,
            bias=False)

        # weight sharing scheme
        # this ensures that the embedding layer and the output layers share weights
        self.tok_embeddings.weight = self.output.weight

        # precompute RoPE frequencies
        self.freqs_cis = precompute_rotary_embeddings(
            params.dim // params.n_heads,
            params.max_seq_len * 2,
            self.device,
            params.rope_theta,)

        # precompute the causal attention mask
        mask = create_causal_mask(params.max_seq_len, self.device)

        self.register_buffer('mask', mask, persistent=True) #persistent ensures this gets saved in the state_dict

        self.criterion = nn.CrossEntropyLoss()

    def forward(self, # specifically for training. this is what you saw in section 1
                tokens: torch.Tensor,
                targets: torch.Tensor):
        bsz, seqlen = tokens.shape
        assert tokens.shape == targets.shape
        assert seqlen == self.max_seq_len

        # initialize the first residual state
        h = self.tok_embeddings(tokens)

        # grab precomputes freqs_cis
        freqs_cis = self.freqs_cis.to(h.device)
        freqs_cis = self.freqs_cis[:seqlen]

        # run the residual state through each layer
        for layer in self.layers:
            h = layer(
                h,
                freqs_cis,
                self.mask,
                start_pos = None,
                training = True
            )

        # norm the final output then get the logits
        h = self.norm(h)
        logits = self.output(h).float()

        loss = self.criterion(
            logits.view(bsz * seqlen, self.vocab_size),
            targets.reshape(bsz * seqlen))

        return logits, loss

    @torch.inference_mode()
    def forward_inference(self,
                          tokens: torch.Tensor,
                          start_pos: int,
                          max_context_window: int,
                         ):
        _bsz, seqlen = tokens.shape
        h = self.tok_embeddings(tokens)
        self.freqs_cis = self.freqs_cis.to(h.device)
        freqs_cis = self.freqs_cis[start_pos : start_pos + seqlen]

        mask = self.mask[:seqlen, :seqlen]
        # When performing key-value caching, we compute the attention scores
        # only for the new sequence. Thus, the matrix of scores is of size
        # (seqlen, cache_len + seqlen), and the only masked entries are (i, j) for
        # j > cache_len + i, since row i corresponds to token cache_len + i.
        mask = torch.hstack(
            [torch.zeros((seqlen, start_pos), device=tokens.device), mask]
        ).type_as(h)
        #print(seqlen)
        #print(start_pos)
        #print( mask.shape)


        for layer in self.layers:
            h = layer(
                h,
                freqs_cis,
                mask,
                start_pos = start_pos
            )
        h = self.norm(h)
        logits = self.output(h).float()
        return logits

    @torch.inference_mode() # no need to keep track of gradients during inference
    def Sampler(
        self,
        logits: torch.Tensor, # shape (batch_size, input_len, vocab_size)
        temperature: float, # controls how boring vs random the outputs should be
        top_p: float, # the maximum cumulative probability of output options we're willing to consider
        top_k: int, # the maximum number of output options we're willing to consider
    ) -> torch.Tensor:
        """
        The Sampler function is responsible for generating token predictions
        It supports temperature scaling, top-p (nucleus) sampling, and top-k sampling
        """
        # Select the last element for each sequence.
        logits = logits[:,-1,:] # (batch_size, input_len, vocab_size) -> (batch_size, vocab_size)

        # Apply temperature scaling
        logits.div_(temperature) # (batch_size, vocab_size) / float -> (batch_size, vocab_size)

        # Calculate probabilities with softmax.
        probs = torch.softmax(logits, dim=-1, dtype=torch.float) # dim=-1 is the vocab_size dimension that we calculate along

        # sort the probabilities to for use in top-p & top-k. both are (batch_size, vocab_size)
        probs_sort, probs_idx = torch.sort(probs, dim=-1, descending=True)

        ### calculating top-p
        # creates same-size tensor of cumulatve probabilities instead of indivdiual probs
        probs_sum = torch.cumsum(probs_sort, dim=-1)
        # mask where 0's are top-p selections & 1's are to be excluded
        top_ps_mask = (probs_sum - probs_sort) > top_p
        # the original probabilities with excluded tokens changed to 0.0
        probs_sort = torch.where(top_ps_mask, 0, probs_sort)

        ### calculating top_k
        # create a shape (vocab_size) tensor that just iterates up by 1's
        top_ks_mask = torch.arange(probs_idx.shape[-1], device=probs_idx.device)
        # expand our mask along the batch_size dimension to become size (batch_size, vocab_size)
        top_ks_mask = top_ks_mask.expand(probs_idx.shape[0], -1)
        # top_ks is a list of integers. we keep whichever entries in top_ks_mask are greater than their corresponding entries in top_ks
        top_ks_mask = top_ks_mask >= top_k

        # we'll be combining top-p with top-k and using whichever gives us fewer tokens. a very conservative approach
        # this trims probs_sort to also fit within our top_k requirement
        probs_sort = torch.where(top_ks_mask, 0, probs_sort)

        # Re-normalization so that total probabilities add up to 1
        probs_sort.div_(probs_sort.sum(dim=-1, keepdim=True))

        # now we rearrange the modified probabilities in probs_sort back to their original order according to probs_idx
        probs = torch.gather(probs_sort, dim=-1, index=torch.argsort(probs_idx, dim=-1))

        # samples from the distribution
        next_token_id = torch.multinomial(probs, num_samples=1)

        return next_token_id # returns the predicted token

    @torch.inference_mode()
    def generate(
        self,
        prompt: str,
        max_gen_len: int = None,
        temperature: float = 0.6, # default value in meta's code
        top_p: float = 0.9, # default value in meta's code
        top_k: int = params.vocab_size, # meta's code doesn't bother with topk
    ) -> str:
        """ Wrapper around sampler() that deals with manipulation of the sequence """

        max_context_window = self.max_seq_len

        # encoding the prompt into token indices
        tokens = self.tokenizer.encode(prompt)

        if max_gen_len is None:
            max_gen_len = self.max_seq_len - len(tokens)
        elif max_gen_len + len(tokens) > self.max_seq_len:
            print(f'capping max_gen_len at max_seq_len={self.max_seq_len} including input\n')
            max_gen_len = self.max_seq_len - len(tokens)

        # turning it into the right tensor shape
        tokens = torch.tensor(tokens, device=self.device)
        tokens = tokens.unsqueeze(0) if len(tokens.shape)==1 else tokens # jic we need to add a batch dimension

        # the offset used for kv caching
        start_pos = max(tokens.shape[1] - max_context_window, 0)

        for i in range(max_gen_len):
            # get the model's output logits and ignore the loss, which would be a NoneType object
            logits = self.forward_inference(
                tokens[:,-max_context_window:],
                start_pos = start_pos,
                max_context_window = max_context_window
            )

            # sample the next token to be used from the logit distribution
            next_token = self.Sampler(
                logits = logits,
                temperature = temperature,
                top_p = top_p,
                top_k = top_k
            )

            # add our new token to the sequence
            tokens = torch.cat((tokens, next_token), dim=1)

            # iterate the offset used in kv caching
            if tokens.shape[1] >= max_context_window:
                start_pos += 1

        # decode our list of tokens to an actual string
        output = self.tokenizer.decode(tokens.squeeze(0).tolist())

        return output

## Data Loader

In [18]:
device_type = device.type

In [19]:
# set up DDP (distributed data parallel).
# torchrun command sets the env variables RANK, LOCAL_RANK, and WORLD_SIZE
ddp = int(os.environ.get('RANK', -1)) != -1 # is this a ddp run?
if ddp:
    # use of DDP atm demands CUDA, we set the device appropriately according to rank
    assert torch.cuda.is_available(), "for now i think we need CUDA for DDP"
    init_process_group(backend='nccl')
    ddp_rank = int(os.environ['RANK'])
    ddp_local_rank = int(os.environ['LOCAL_RANK'])
    ddp_world_size = int(os.environ['WORLD_SIZE'])
    device = f'cuda:{ddp_local_rank}'
    torch.cuda.set_device(device)
    master_process = ddp_rank == 0 # this process will do logging, checkpointing etc.
else:
    # vanilla, non-DDP run
    ddp_rank = 0
    ddp_local_rank = 0
    ddp_world_size = 1
    master_process = True
    # attempt to autodetect device
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda"
    elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
        device = "mps"
print(f"using device: {device}")

using device: cuda


In [20]:
#DataLoader code
def load_tokens(filename):
    npt = np.load(filename)
    npt = npt.astype(np.int32)
    ptt = torch.tensor(npt, dtype=torch.long)
    return ptt


class DataLoaderLite:
    def __init__(self, B, T, process_rank, num_processes, split):
        self.B = B
        self.T = T
        self.process_rank = process_rank
        self.num_processes = num_processes
        assert split in {'train', 'val'}

        # get the shard filenames
        data_root = "edu_fineweb10B"
        shards = os.listdir(data_root)
        shards = [s for s in shards if split in s]
        shards = sorted(shards)
        shards = [os.path.join(data_root, s) for s in shards]
        self.shards = shards
        assert len(shards) > 0, f"no shards found for split {split}"
        #if master_process:
        print(f"found {len(shards)} shards for split {split} on device {device}")
        self.reset()

    def reset(self):
        # state, init at shard zero
        self.current_shard = 0
        self.tokens = load_tokens(self.shards[self.current_shard])
        self.current_position = self.B * self.T * self.process_rank

    def next_batch(self):
        B, T = self.B, self.T
        buf = self.tokens[self.current_position : self.current_position+B*T+1]
        x = (buf[:-1]).view(B, T) # inputs
        y = (buf[1:]).view(B, T) # targets
        # advance the position in the tensor
        self.current_position += B * T * self.num_processes
        # if loading the next batch would be out of bounds, advance to next shard
        if self.current_position + (B * T * self.num_processes + 1) > len(self.tokens):
            self.current_shard = (self.current_shard + 1) % len(self.shards)
            self.tokens = load_tokens(self.shards[self.current_shard])
            self.current_position = B * T * self.process_rank
        return x, y

## Training Utils

In [21]:
B = params.max_batch_size #16
T = params.max_seq_len #1024
train_loader = DataLoaderLite(B=B, T=T, process_rank=ddp_rank, num_processes=ddp_world_size, split="train")
val_loader = DataLoaderLite(B=B, T=T, process_rank=ddp_rank, num_processes=ddp_world_size, split="val")

found 99 shards for split train on device cuda
found 1 shards for split val on device cuda


In [22]:
tokenizer = tiktoken.get_encoding("gpt2")
eot = tokenizer._special_tokens['<|endoftext|>'] # end of text token

In [23]:
#instantiate model
model = LLM(params, tokenizer, device)
model = model.to(device)

#torch.compile
model = torch.compile(model)  #use torch.compile to speed things up

#wrap the model in DDP
if ddp:
    model = DDP(model, device_ids=[ddp_local_rank])

if master_process:
    s = sum(p.numel() for p in model.parameters())
    print(f"{s} parameter model trained on 10 Billion tokens")

114150144 parameter model trained on 10 Billion tokens


In [24]:
@torch.no_grad()
def estimate_val_loss(model, batch_size, eval_iters = 10): # to estimate loss during the training loop
    out = {}
    model.eval() # sets model to eval mode
    #for split in ['train', 'val']:
    for split in ['val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            #X, Y = get_batch(split, batch_size)
            #logits, loss = model(X, targets=Y)
            #losses[k] = loss.item()
            #dl = get_dataloader(split, batch_size)
            #for s, t in dl:
            s, t = val_loader.next_batch()
            s, t = s.to(device), t.to(device)
            # if the GPU supports BFLOAT, autocast to BF16
            if (params.bfloat_supported):
              with torch.autocast(device_type=device_type, dtype=torch.bfloat16):
                logits, loss = model(s, targets=t)
                losses[k] = loss.item()
            else:
              logits, loss = model(s, targets=t)
              losses[k] = loss.item()

        out[split] = losses.mean()
    model.train() # just resets to training mode
    return out

In [25]:
#***************************************************
# how long we want to train for
#***************************************************
max_iters = 15000 #2000
# how often we want to check & see how our loss is doing
eval_interval = 50

# Warmup setup
warmup_iters = 500  # Number of warmup iterations
warmup_factor = 1e-3  # Warmup factor (initial learning rate is multiplied by this factor)

# create a PyTorch optimizer
lr_init = 6e-4
lr_final = lr_init * 0.1 # Minimum learning rate
weight_decay = 0.02
optimizer = torch.optim.AdamW(model.parameters(), lr=lr_init, weight_decay=weight_decay)

In [26]:
def lr_lambda(current_iter):
    if current_iter < warmup_iters:
        # Warmup phase
        return warmup_factor + (1 - warmup_factor) * current_iter / warmup_iters
    else:
        # Cosine decay phase with minimum learning rate
        decay_iters = max_iters - warmup_iters
        cosine_decay = 0.5 * (1 + math.cos(math.pi * (current_iter - warmup_iters) / decay_iters))
        return max(cosine_decay, lr_final / lr_init)

In [27]:
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

## Train

In [28]:
start_time = time.time()

# Enable anomaly detection. uncomment these lines if you need to do extensive debugging
#torch.autograd.set_detect_anomaly(True)

for iter in range(max_iters):
    loss_accum = 0.0 #this is just for display purposes
    model.train()
    for microstep in range(params.grad_accum_steps):
        #dl = get_dataloader('train', params.max_batch_size)
        #for s, t in dl:
        s, t = train_loader.next_batch()
        s, t = s.to(device), t.to(device)
        #This might not be the official way to do this. If thing break in the future, look here
        if ddp:
            #model.require_backward_grad_sync = (microstep == params.grad_accum_steps - 1)
            if (microstep == params.grad_accum_steps - 1):
                #print("synching")
                model.require_backward_grad_sync = True
        #if bfloat16 is supported, use it
        if (params.bfloat_supported):
            with torch.autocast(device_type=device_type, dtype=torch.bfloat16):
                logits, loss = model(s, targets=t)
        else:
            logits, loss = model(s, targets=t)
        loss = loss / params.grad_accum_steps
        loss.backward()
        loss_accum += loss.detach()
        norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  #clip gradients
        #optimizer.step() # update weights
        #optimizer.zero_grad() #REMEMBER TO ZERO THE GRADIENTS!!!
    if ddp:
        dist.all_reduce(loss_accum, op=dist.ReduceOp.AVG)

    optimizer.step() # update weights
    optimizer.zero_grad() #REMEMBER TO ZERO THE GRADIENTS!!!
    # Update the learning rate
    scheduler.step()

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        current_time = time.time()
        elapsed_time = current_time - start_time

        if master_process:
            #only estimate Val loss for the Master process
            losses = estimate_val_loss(model, params.max_batch_size)
            current_lr = optimizer.param_groups[0]['lr']
            print(f"step {iter:04d}: lr {current_lr:.6f}, train loss {loss_accum:.4f}, val loss {losses['val']:.4f}, time elapsed: {elapsed_time:.2f} seconds")

step 0000: lr 0.000002, train loss 11.0041, val loss 10.9982, time elapsed: 36.14 seconds
step 0050: lr 0.000062, train loss 8.6184, val loss 8.6349, time elapsed: 77.28 seconds
step 0100: lr 0.000122, train loss 7.6952, val loss 7.5110, time elapsed: 103.09 seconds
step 0150: lr 0.000182, train loss 7.7130, val loss 7.0836, time elapsed: 128.93 seconds
step 0200: lr 0.000242, train loss 6.6360, val loss 6.7255, time elapsed: 154.81 seconds
step 0250: lr 0.000301, train loss 6.3251, val loss 6.4910, time elapsed: 180.71 seconds
step 0300: lr 0.000361, train loss 6.1571, val loss 6.2282, time elapsed: 206.61 seconds
step 0350: lr 0.000421, train loss 5.8220, val loss 6.1237, time elapsed: 232.52 seconds
step 0400: lr 0.000481, train loss 5.8494, val loss 5.8538, time elapsed: 258.43 seconds
step 0450: lr 0.000541, train loss 5.7364, val loss 5.7070, time elapsed: 284.35 seconds
step 0500: lr 0.000600, train loss 5.6161, val loss 5.5538, time elapsed: 310.27 seconds
step 0550: lr 0.00060

In [30]:
#Save the model
if master_process:
    print("SAVING MODEL")
    raw_model = model.module if ddp else model # always contains the "raw" unwrapped model

    input_str = "Hello, I am Mr. LLM. I "
    output = raw_model.generate(
        input_str,
        max_gen_len = 50 + len(input_str),
        temperature = 0.6,
        top_p = 0.9,
        top_k = 32,
    )
    print("----------------")
    print(output)

    input_str = "These are the symptoms of HIV. "
    output = raw_model.generate(
        input_str,
        max_gen_len = 50 + len(input_str),
        temperature = 0.6,
        top_p = 0.9,
        top_k = 32,
    )
    print("----------------")
    print(output)

    input_str = "Here is a story about a pirate and a ninja. "
    output = raw_model.generate(
        input_str,
        max_gen_len = 200 + len(input_str),
        temperature = 0.6,
        top_p = 0.9,
        top_k = 32,
    )
    print("----------------")
    print(output)

    os.makedirs("working", exist_ok=True)
    os.makedirs("download", exist_ok=True)
    torch.save(raw_model.state_dict(), './working/best_model_state-114m.bin')

SAVING MODEL
----------------
Hello, I am Mr. LLM. I __________
The first thing to do is to read the text in the dictionary. I want to read the text in the dictionary. I want to read the text in the dictionary. I want to read the text in the dictionary. I want to read the text in the dictionary. I want to read the text in the dictionary. I want to read the text
----------------
These are the symptoms of HIV. 
The symptoms of HIV may vary, but they may vary depending on the person's age, sex, and other factors.
People with HIV are more likely to develop HIV if they have a family history of HIV, which is the most common reason for HIV infection.
Some people with HIV may have a family history of HIV.
The HIV infection may affect a person's ability to function, or
----------------
Here is a story about a pirate and a ninja. 
The story goes on to explain how he was a great warrior and how he was a very successful pirate. 
He was a good man and he was a good man. He was a good man and he was

In [31]:
#exit gracefully
if ddp:
    destroy_process_group()